In [1]:
import pandas as pd
import numpy as np
from __future__ import division
from pyomo.environ import *
import pickle
import time

Preprocessing

In [2]:

all_data=pd.read_csv('../data/Timeseries_data_SP500.csv')

all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

date_list=list(set(list(all_data['date'])))
date_list.sort()

dic_data = {k: v for k, v in all_data.groupby('date')}

results_=pd.read_csv('../data/results_template.csv')
results_=results_.rename(columns={'DATE':'date','SEDOL':'sedol','WEIGHTS':'w','FOUR_WEEKLY_RETURN':'r'})
results_['date']=pd.to_datetime(results_['date'])
dic_results_={k: v for k, v in results_.groupby('date')}

dic_r_all={}
for i in date_list:
    rr={}
    for j,k in zip(list(dic_results_[i]['sedol']),list(dic_results_[i]['r'])):
        rr[j]=k
    dic_r_all[i]=rr

def preprocessing(date):
    
    using_dic = dic_data[pd.to_datetime(date)]
    using_dic['index']=list(using_dic.index)

    asset_list = []
    for i in using_dic["sedol"]:
        asset_list.append(i)


    dic_sedol_as = {using_dic["sedol"][i] : using_dic["as"][i] for i in using_dic.index}

    dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

    dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

    dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }

    dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

    dic_aseet_MCAP = {using_dic["sedol"][i] : using_dic["mq"][i] for i in using_dic.index}
    
    dic_aseet_sector = {using_dic["sedol"][i] : using_dic["sector"][i] for i in using_dic.index}
    
    for i in using_dic.index:
        dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
        dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])



    #risk:cov_mat
    date_str=str(date)[:10]
    risk_data=pd.read_csv('../data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    #         return risk_data,risk_sedol,risk_mat

    
    alpha = []
    
    dic_risk_index = {}
    index_ri = 0

    for i in risk_sedol:
        dic_risk_index.update({i:index_ri})
        index_ri += 1
        alpha.append(-1*dic_sedol_as[i])

    dic_r={dic_results_[date]["sedol"][i] : dic_results_[date]["r"][i] for i in using_dic.index}

    return using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector,dic_risk_index

Modeling (Nonlinear_QCQP)

In [2]:
############# Settings ##############
n=0  # month
date=date_list[n]
using_dic,NEW_stocks,dic_alpha,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector,dic_risk_index=preprocessing(date)

omega={}
for i in NEW_stocks:
    omega[i]={}
    for j in NEW_stocks:
        omega[i][j]=risk_mat[NEW_stocks.index(i)][NEW_stocks.index(j)]

model = ConcreteModel()


############## set #################

model.I = Set(initialize = NEW_stocks)
model.J = Set(initialize = NEW_stocks)
model.S = Set(initialize = list(dic_sector.keys()))
model.K = Set(initialize = list(dic_MCAP.keys()))


############# Decision Variables ###############

model.d = Var([i for i in model.I], domain=Reals, bounds=(-0.05,0.05))
model.w = Var([i for i in model.I], domain=NonNegativeReals, bounds=(0,1))
model.y = Var([i for i in model.I], domain=Binary)


############# Constraints ##############

def constraint1(model , i) :
    return model.d[i] == model.w[i] - dic_bench[i]
model.con1 = Constraint(model.I, rule=constraint1)

def constraint4(model) :
    return sum(model.w[i] for i in model.I) == 1
model.con4 = Constraint(rule=constraint4)

def constraint6(model, s) :
    return -0.1 <= sum(model.d[i] for i in dic_sector[s]) <= 0.1 
model.con6 = Constraint(model.S, rule=constraint6)    

def constraint7(model, k) :
    return -0.1 <= sum(model.d[i] for i in dic_MCAP[k]) <= 0.1 
model.con7 = Constraint(model.K, rule=constraint7)    

def constraint8(model):
    return -0.1 <= sum(model.d[i]*dic_beta[i] for i in model.I) <= 0.1

def constraint9_1(model, i) :
    return model.y[i] >= model.w[i]
model.con9_1 = Constraint(model.I, rule=constraint9_1)    

def constraint9_2(model, i) :
    return model.y[i] <= model.w[i]+0.999
model.con9_2 = Constraint(model.I, rule=constraint9_2)    

def constraint9_3(model) :
    return 50 <= sum(model.y[i] for i in model.I) <= 70
model.con9_3 = Constraint(rule=constraint9_3)    

def constraint10(model) :
    return 0.6 <= 0.5 * sum(abs(model.d[i]) for i in model.I) <= 1.
model.con10 = Constraint(rule=constraint10)    

def constraint11(model) :
    return 0.0025 <= (sum(model.d[i]*omega[i][j]*model.d[j] for i in model.I for j in model.J)) <= 0.01
model.con11 = Constraint(rule=constraint11)   


############ Objective Function #############

def objective_rule(model):
    return sum(-model.d[i] * dic_alpha[i] for i in model.I) +sum(model.d[i]*omega[i][j]*model.d[j] for i in model.I for j in model.J) 


model.objective = Objective(rule=objective_rule, sense=minimize)



Solve (couenne)

In [3]:
######### Execute(Solve) ##########
start=time.time()
solver = SolverFactory('couenne', executable='./couenne')
solution = solver.solve(model,tee=True)
time = time.time() - start


######## Save Results #########

w = model.w.get_values()
d = model.d.get_values()
y = model.y.get_values()


with open('Results/T1(couenne).txt', 'wb') as f:
    pickle.dump(w, f)
    pickle.dump(d, f)
    pickle.dump(y, f)
    pickle.dump(solution, f)
    pickle.dump(time,f)



############ Trial : 1 #############
Couenne 0.5.6 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Reformulating problem: 13.8 seconds
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 0.0024845502      105 59.19317
NLP0014I             2         OPT 0.0026787841       37 11.891635
Couenne: new cutoff value 2.6787840688e-03 (108.152 seconds)
Loaded instance "/tmp/tmpawsosxu8.pyomo.nl"
Constraints:         1495
Variables:           1476 (492 integer)
Auxiliaries:        95892 (1 integer)

Coin0506I Presolve 383012 (-493) rows, 96364 (-1004) columns and 1336291 (-1496) elements
Clp0006I 0  Obj 0 Primal inf 64.346371 (495) Dual inf 1.0755131e+15 (96364)
Clp0006I 1000  Obj 0 Primal inf 64.346371 (495) Dual inf 5.6236277e+14 (1086)
Clp0006I 2000  Obj 0 Primal inf 64.346371 (495) Dua

NLP Heuristic: time limit reached.
Clp0006I 0  Obj 0.002484374 Dual inf 0.0012810545 (1804)
Clp0006I 0  Obj 0.002484374 Dual inf 0.0012810545 (1804)
Clp0006I 200  Obj 0.0024843278 Dual inf 9.3750505e-05 (895)
Clp0006I 400  Obj 0.0024842895
Clp0006I 600  Obj 0.0024842863
Clp0006I 800  Obj 0.0024842863
Clp0006I 1000  Obj 0.0024842855
Clp0006I 1200  Obj 0.0024842831
Clp0000I Optimal - objective value 0.0024842831
Optimality Based BT: 0 improved bounds
Probing: 0 improved bounds
NLP Heuristic: no solution.
Cbc0031I 18 added rows had average density of 2
Cbc0013I At root node, 18 cuts changed objective from 0.0024842831 to 0.0024842831 in 6 passes
Cbc0014I Cut generator 0 (Couenne convexifier cuts) - 52 row cuts average 2.0 elements, 1 column cuts (1 active)
Cbc0010I After 0 nodes, 1 on tree, 0.0026787841 best solution, best possible 0.0024842831 (3037.72 seconds)
Optimality Based BT: 1 improved bounds
Optimality Based BT: 0 improved bounds
Optimality Based BT: 0 improved bounds
Optimality 

Cbc0010I After 6200 nodes, 3080 on tree, 0.0026787841 best solution, best possible 0.0024842831 (142460.97 seconds)
Cbc0010I After 6300 nodes, 3130 on tree, 0.0026787841 best solution, best possible 0.0024842831 (143650.29 seconds)
Cbc0010I After 6400 nodes, 3180 on tree, 0.0026787841 best solution, best possible 0.0024842831 (147328.97 seconds)
Cbc0010I After 6500 nodes, 3230 on tree, 0.0026787841 best solution, best possible 0.0024842831 (149169.87 seconds)
Cbc0010I After 6600 nodes, 3280 on tree, 0.0026787841 best solution, best possible 0.0024842831 (150212.79 seconds)
Cbc0010I After 6700 nodes, 3330 on tree, 0.0026787841 best solution, best possible 0.0024842831 (151554.42 seconds)
Cbc0010I After 6800 nodes, 3380 on tree, 0.0026787841 best solution, best possible 0.0024842831 (152565.38 seconds)
Cbc0010I After 6900 nodes, 3430 on tree, 0.0026787841 best solution, best possible 0.0024842831 (153589.97 seconds)
Cbc0010I After 7000 nodes, 3480 on tree, 0.0026787841 best solution, bes

SystemExit: -2

/opt/anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
